In [ ]:
from __future__ import annotations
import json
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import (
    train_test_split, StratifiedKFold, GridSearchCV
)
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score,
    confusion_matrix, ConfusionMatrixDisplay
)
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier,
    HistGradientBoostingClassifier,
    StackingClassifier
)
from sklearn.inspection import permutation_importance
import joblib

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
ARTIFACTS_DIR = Path("artifacts")
FIGURES_DIR = ARTIFACTS_DIR / "figures"
ARTIFACTS_DIR.mkdir(exist_ok=True)
FIGURES_DIR.mkdir(exist_ok=True)

In [ ]:
df = pd.read_csv("S06-hw-dataset-03.csv")

print("Размер:", df.shape)

print("\nПервые 5 строк:\n", df.head())

print("\nРаспределение target:")
print(df["target"].value_counts(normalize=True).sort_index())

X = df.drop(columns=["id", "target"])
y = df["target"]

print("\nПризнаки:", X.columns.tolist())
print("Типы:", X.dtypes.unique())
print("Пропусков:", X.isnull().sum().sum())
feature_names = X.columns.tolist()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=RANDOM_STATE,
    stratify=y
)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

In [ ]:
def compute_metrics(y_true, y_pred, y_proba=None):
    return {
        "accuracy": float(accuracy_score(y_true, y_pred)),
        "f1_macro": float(f1_score(y_true, y_pred, average="macro")),
        "roc_auc_ovr": float(roc_auc_score(y_true, y_proba, multi_class="ovr")) if y_proba is not None else None
    }

def evaluate_on_test(model, X_test, y_test, label):
    pred = model.predict(X_test)
    proba = model.predict_proba(X_test) if hasattr(model, "predict_proba") else None
    metrics = compute_metrics(y_test, pred, proba)
    metrics["model"] = label
    return metrics

In [ ]:
results = []
best_models = {}
search_summaries = {}

# Dummy
dummy = DummyClassifier(strategy="most_frequent", random_state=RANDOM_STATE)
dummy.fit(X_train, y_train)
results.append(evaluate_on_test(dummy, X_test, y_test, "Dummy(most_frequent)"))
best_models["Dummy(most_frequent)"] = dummy

# Logistic Regression
lr_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LogisticRegression(max_iter=1000, random_state=RANDOM_STATE))
])
lr_grid = {"lr__C": [0.1, 1.0, 10.0]}
lr_search = GridSearchCV(lr_pipe, lr_grid, cv=cv, scoring="roc_auc_ovr")
lr_search.fit(X_train, y_train)
best_models["LogReg(scaled)"] = lr_search.best_estimator_
search_summaries["LogReg(scaled)"] = {
    "best_params": lr_search.best_params_,
    "best_cv_roc_auc": float(lr_search.best_score_)
}
results.append(evaluate_on_test(lr_search.best_estimator_, X_test, y_test, "LogReg(scaled)"))

In [ ]:
def run_search(label, estimator, param_grid):
    search = GridSearchCV(estimator, param_grid, cv=cv, scoring="roc_auc_ovr", n_jobs=-1)
    search.fit(X_train, y_train)
    best = search.best_estimator_
    best_models[label] = best
    search_summaries[label] = {
        "best_params": search.best_params_,
        "best_cv_roc_auc": float(search.best_score_)
    }
    results.append(evaluate_on_test(best, X_test, y_test, label))

# Decision Tree
dt = DecisionTreeClassifier(random_state=RANDOM_STATE)
run_search("DecisionTree", dt, {
    "max_depth": [3, 5, 8, 10],
    "min_samples_leaf": [1, 5, 10, 20],
    "ccp_alpha": [0.0, 0.001, 0.01]
})

# Random Forest
rf = RandomForestClassifier(n_estimators=500, random_state=RANDOM_STATE, n_jobs=-1)
run_search("RandomForest", rf, {
    "max_depth": [None, 6, 10],
    "min_samples_leaf": [1, 5],
    "max_features": ["sqrt", 0.5]
})

# HistGradientBoosting
hgb = HistGradientBoostingClassifier(random_state=RANDOM_STATE, early_stopping=True)
run_search("HistGradientBoosting", hgb, {
    "learning_rate": [0.05, 0.1],
    "max_leaf_nodes": [15, 31],
    "max_depth": [3, None]
})

In [ ]:
df_results = pd.DataFrame(results)
df_results.to_json(ARTIFACTS_DIR / "metrics_test.json", orient="records", indent=2)
with open(ARTIFACTS_DIR / "search_summaries.json", "w", encoding="utf-8") as f:
    json.dump(search_summaries, f, indent=2, ensure_ascii=False)

In [ ]:
best_row = df_results.loc[df_results["roc_auc_ovr"].idxmax()]
best_label = best_row["model"]
best_model = best_models[best_label]

# Confusion Matrix
cm = confusion_matrix(y_test, best_model.predict(X_test))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1, 2])
disp.plot()
plt.savefig(FIGURES_DIR / "confusion_matrix.png")
plt.show()

# Permutation Importance
perm = permutation_importance(
    best_model, X_test, y_test,
    n_repeats=5, random_state=RANDOM_STATE,
    scoring="roc_auc_ovr"
)
idx = np.argsort(perm.importances_mean)[::-1][:10]
plt.figure(figsize=(8, 4))
plt.bar(range(len(idx)), perm.importances_mean[idx])
plt.xticks(range(len(idx)), [feature_names[i] for i in idx], rotation=45, ha="right")
plt.title(f"{best_label}: Permutation Importance (top-10)")
plt.tight_layout()
plt.savefig(FIGURES_DIR / "permutation_importance.png")
plt.show()

In [ ]:
joblib.dump(best_model, ARTIFACTS_DIR / "best_model.joblib")
meta = {
    "best_model_label": best_label,
    "test_metrics": best_row.to_dict(),
    "search_summary": search_summaries.get(best_label, None),
    "random_state": RANDOM_STATE
}
with open(ARTIFACTS_DIR / "best_model_meta.json", "w", encoding="utf-8") as f:
    json.dump(meta, f, indent=2, ensure_ascii=False)